<a href="https://colab.research.google.com/github/IrisFDTD/AI-UNIZAR-course/blob/main/regression/ai_unizar_course_regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">*Introducción a la inteligencia artificial: redes neuronales avanzadas con Tensorflow-Keras*</span> por <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">Sergio Gutiérrez Rodrigo (sergut@unizar.es)</span> se distribuye bajo una <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Licencia Creative Commons Atribución-NoComercial-CompartirIgual 4.0 Internacional</a>.

```
Sergio G Rodrigo
Departamento de Física Aplicada
Universidad de Zaragoza
Instituto de Nanociencia y Materiales de Aragón (INMA)
C/ Pedro Cerbuna, 12, 50009, Zaragoza, España
```






# **Advanced Neural Networks with TensorFlow-Keras**

---
# **Regression with Dense Neural Networks (DNN)**
---
A neural network as a subrogate for the function $y(x)=exp(-x)$, trained in the range $x \in [0,x_{max}]$.

## Main libraries

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Input,Dense
from keras import Model
from keras.losses import MeanSquaredError as mse_losses
from keras.metrics import MeanSquaredError as mse_metrics
from keras.optimizers import Adam,SGD

import numpy as np
import matplotlib.pyplot as plt

## Training data

In [ ]:
n_train = 20
xmin = 0
xmax = 4

# Definition of the function domain
x_train = np.linspace(xmin,xmax,n_train)

# The true y(x) (used for training)
y_train = np.exp(-x_train)

## Hyperparameters

In [ ]:
# Hiperparameters
epochs = 100
lr = 0.001

## Model (version 1)

In [ ]:
# Definition of the the model
model_1 = Sequential()
model_1.add(Input(shape=(1,)))
model_1.add(Dense(50, activation='elu'))
model_1.add(Dense(50, activation='elu'))
model_1.add(Dense(1, activation=None))

## Model (version 2)

In [ ]:
model_2 = Sequential([Input(shape=(1,)),
                   Dense(50, activation='elu'),
                   Dense(50, activation='elu'),
                   Dense(1, activation=None)])

## Model (version 3): Keras functional application programming interface (API)

In [ ]:
input = Input(shape=(1,))
x = Dense(50, activation='elu')(input)
x = Dense(50, activation='elu')(x)
output = Dense(1,activation=None)(x)
model_3 = Model(input,output)

## Metrics, optimizer and loss

In [ ]:
# Choose the model!!
model = model_1

# Definition of the metrics, optimizer and loss
loss = mse_losses
metrics = mse_metrics
optimizer = Adam(learning_rate=lr)

model.compile(loss=loss,
          optimizer=optimizer,
          metrics=[metrics])

## Run the model

In [ ]:
model.summary()
history = model.fit(x_train, y_train,batch_size=1,epochs=epochs)

##  Evolution of losses during training

In [ ]:
# summarize history for loss and metris
plt.rcParams['figure.dpi'] = 100
plt.plot(history.history['loss'],
         color='magenta',
         label='Loss')
plt.plot(history.history['mean_squared_error'],
         color='cyan',
         label='MSE',linestyle='--')
plt.yscale("log")
plt.xlabel('epochs')
plt.legend(loc='upper right')
plt.show()

## Validation: $y(x)$ vs $y_{NN}(x)$

In [ ]:
n = 500
x=np.linspace(0,4,n)
y_exact=np.exp(-x)
y_NN=model.predict(x)

# Plot the results
plt.rcParams['figure.dpi'] = 100
plt.plot(x, y_exact, color="black",linestyle='solid',
                     linewidth=2.5,label="$y(x)$ analytical")
plt.plot(x, y_NN, color="red",linestyle='dashed',
                     linewidth=2.5, label="$y_{NN}(x)$")
plt.legend()
plt.xlabel("x")
plt.show()

# Extract weights and bias: a custom feed-forward model

In [ ]:
# Obtain all model weights and bias
model.weights

In [ ]:
# Let's use Keras functional API: a feed_forward function
def feed_forward(x):
  prediction = []
  for xi in x:
    # Each digit, xi, must be reshaped
    # Cast a to float32 to match the expected type (corrected by IA!!!)
    a=np.array([[xi]]).astype(np.float32)
    #a=np.array([[xi]])  # Working a year later (2023)
    # For each layer in model we use the layer properties to
    # feedforward
    for layer in model.layers[0:]:
       #print("Layer %s" %(layer.name))
       #print(a.shape)
       w=layer.kernel
       b=layer.bias
       #print(w.shape,b.shape)
       z=a@w+b
       a=layer.activation(z)
    prediction.append(a)
  return prediction

# feed_forward in pythonic style
def feed_forward_pythonic(x):
  a=np.reshape(x,(x.shape[0],1)).astype(np.float32)
  print(a.shape)
  for layer in model.layers[0:]:
      w,b=layer.kernel,layer.bias
      a=layer.activation(a@w+b)
  return a

In [ ]:
n = 500
x=np.linspace(0,4,n)
y_exact=np.exp(-x)
y_NN=feed_forward_pythonic(x)
y_NN=np.squeeze(y_NN) # Remove trivial dimensions

# Plot the results
plt.rcParams['figure.dpi'] = 100
plt.plot(x, y_exact, color="black",linestyle='solid',
                     linewidth=2.5,label="$y(x)$ analytical")
plt.plot(x, y_NN, color="red",linestyle='dashed',
                     linewidth=2.5, label="$y_{NN}(x)$")
plt.legend()
plt.xlabel("x")
plt.show()